In [1]:
from deepview import DeepView
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
# ---------------------------
import demo_utils as demo

%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
# matplotlib qt seems to be a bit buggy with notebooks, so we execute it multiple times
%matplotlib qt

## Load CIFAR10 and a some models

This notebook tests the DeepView framework on different classifiers

 * ResNet-20 on CIFAR10
 * DecisionTree on MNIST
 * RandomForest on MNIST
 * KNN on MNIST

In [5]:
# device will be detected automatically
# Set to 'cpu' or 'cuda:0' to set the device manually
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

testset = demo.make_cifar_dataset()
torch_model = demo.create_torch_model(device)

digits_X, digits_y = demo.make_digit_dataset()
decision_tree = demo.create_decision_tree(digits_X, digits_y, max_depth=10)
random_forest = demo.create_random_forest(digits_X, digits_y, n_estimators=100)
kn_neighbors = demo.create_kn_neighbors(digits_X, digits_y, k=10)

Files already downloaded and verified
Created decision tree
 * Depth:		 10
 * Dataset:		 MNIST
 * Train score:		 0.9833055091819699
Created random forest
 * No. of Estimators:	 100
 * Dataset:		 MNIST
 * Train score:		 1.0
Created knn classifier
 * No. of Neighbors:	 10
 * Dataset:		 MNIST
 * Train score:		 0.9855314412910406


In [100]:
import warnings
warnings.warn("Bla")

/media/luca/a87b7b03-b48e-49f3-903a-66c8d0de2388/python_projects/virt_envs/tf2_venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Bla
  


## Usage Instructions

 1. Create a wrapper funktion like ```pred_wrapper``` which receives a numpy array of samples and returns according class probabilities from the classifier as numpy arrays
 2. Initialize DeepView-object and pass the created method to the constructor
 3. Run your code and call ```add_samples(samples, labels)``` at any time to add samples to the visualization together with the ground truth labels.
    * The ground truth labels will be visualized along with the predicted labels
    * The object will keep track of a maximum number of samples specified by ```max_samples``` and it will throw away the oldest samples first
 4. Call the ```show``` method to render the plot

The following parameters must be specified on initialization:


| <p align="left">Variable    | <p align="left">Meaning             |
|----------------------|-------------------|
| <p align="left">```batch_size```    | <p align="left">Batch size to use when calling the classifier |
| <p align="left">```pred_wrapper```    | <p align="left">To enable DeepView to call the classifier |
| <p align="left">```max_samples```      | <p align="left">The maximum amount of samples that DeepView will keep track of |
| <p align="left">```data_shape```         | <p align="left">Shape of the input data (complete shape; for images, include the channel dimension) |
| <p align="left">```n```     | <p align="left">Number of interpolations for distance calculation of two images. |
| <p align="left">```lam```     | <p align="left">Weights the euclidian distance-component against the discriminative fisher distance. A high lambda puts emphasis on the euclidian distance, so structural information of the data will be more relevant for distance computation. |
| <p align="left">```resolution```       | <p align="left">x- and y- Resolution of the decision boundary plot |
| <p align="left">```cmap```             | <p align="left">Name of the colormap that should be used in the plots. |

## Demo with Torch model

In [17]:
# softmax operation to use in pred_wrapper
softmax = torch.nn.Softmax(dim=-1)

# this is the prediction wrapper, it encapsulates the call to the model
# and does all the casting to the appropriate datatypes
def pred_wrapper(x):
    with torch.no_grad():
        x = np.array(x, dtype=np.float32)
        x = np.transpose(x, [0, 3, 1, 2])
        tensor = torch.from_numpy(x).to(device)
        logits = torch_model(tensor)
        probabilities = softmax(logits).cpu().numpy()
    return probabilities

# the classes in the dataset to be used as labels in the plots
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# --- Deep View Parameters ----
batch_size = 512
max_samples = 500
data_shape = (3, 32, 32)
n = 3
lam = .64
resolution = 100
cmap = 'tab10'
title = 'ResNet-20 - CIFAR10'

deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    data_shape, n, lam, resolution, cmap, title=title)

In [18]:
n_samples = 150
sample_ids = np.random.choice(len(testset), n_samples)
X = np.array([ testset[i][0] for i in sample_ids ])
Y = np.array([ testset[i][1] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()


print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/media/luca/a87b7b03-b48e-49f3-903a-66c8d0de2388/python_projects/virt_envs/tf2_venv/lib/python3.6/site-packages/umap/umap_.py:1503: UserWarning: using precomputed metric; transform will be unavailable for new data and inverse_transform will be unavailable for all data
  "using precomputed metric; transform will be unavailable for new data and inverse_transform "


Computing decision regions ...
Time to calculate visualization for 150 samples: 23.58 sec


## Add new samples to the visualization

In [21]:
n_new = 200

sample_ids = np.random.choice(len(testset), n_new)
X = np.array([ testset[i][0] for i in sample_ids ])
Y = np.array([ testset[i][1] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to add %d samples to visualization: %.2f sec' % (n_new, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to add 200 samples to visualization: 82.53 sec


### Example output

As the plot is updatable, it is shown in a separate Qt-window. With the CIFAR-data and the model loaded above, the following plot was produced after 200 samples where added:

**Hyperparameters:**
n = 10
lam = 0.2
resolution = 100

![sample_plot](https://user-images.githubusercontent.com/30961397/72370639-fbab6f00-3702-11ea-98f4-0dc7335777fc.png)

## Tuning the $\lambda$-Hyperparameter

> The $\lambda$-Hyperparameter weights the euclidian distance component.
> When the visualization doesn't show class-clusters, **try a smaller lambda** to put more emphasis on the discriminative distance component that considers the class.
> A smaller $\lambda$ will pull the datapoints further into their class-clusters.
> Therefore, a **too small $\lambda$** can lead to collapsed clusters that don't represent any structural properties of the datapoints. Of course this behaviour also depends on the data and how well the label corresponds to certain structural properties.

Due to separate handling of euclidian and class-discriminative distances, the $\lambda$ parameter can easily be adjusted. Distances don't need to be recomputed, only the embeddings and therefore also the plot of the decision boundary.

In [10]:
deepview.set_lambda(.7)
deepview.show()

Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...


## Compare performance

For this test, DeepView was run on a GPU (GTX 2060 6GB).
Adding samples may be a bit more time consuming, then just running DeepView on the desired amount of samples to be visualized. This is because the decision boundaries must be calculated twice with a similar time complexity. However, the step of adding 100 samples to 100 existing samples takes less time then computing it from scratch for 200 samples. This is because distances were already computed for half of the samples and can be reused.

| <p align="left">Szenario | Time |
| -------- | ---- |
| <p align="left">From scratch for 100 samples | 31.20 sec |
| <p align="left">Adding 100 samples (100 already added) | 66.89 sec |
| <p align="left">From scratch for 200 samples | 71.16 sec |
| <p align="left">200 samples when adding 100 samples in two steps | 98.19 sec |

In [8]:
deepview.reset()

n_samples = 200
sample_ids = np.random.choice(len(testset), n_samples)
X = np.array([ testset[i][0] for i in sample_ids ])
Y = np.array([ testset[i][1] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 67.94 sec


In [19]:
deepview.close()

## Demo with RandomForest

In [119]:
def visualization(image, point2d, pred, label=None):
    f, a = plt.subplots()
    a.set_title('Prediction: %d' % pred)
    a.imshow(image.reshape(8,8))

In [126]:
pred_wrapper = DeepView.create_simple_wrapper(random_forest.predict_proba)

# the digit dataset is used, so classes are [0..9]
classes = np.arange(10)

# --- Deep View Parameters ----
batch_size = 64
max_samples = 500
sample_shape = (64,)
n = 10
lam = 0.5
resolution = 100
cmap = 'tab10'
title = 'RandomForest - MNIST'

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, sample_shape, 
                    n, lam, resolution, cmap, title=title, data_viz=visualization)

# add data samples
n_samples = 50
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/media/luca/a87b7b03-b48e-49f3-903a-66c8d0de2388/python_projects/virt_envs/tf2_venv/lib/python3.6/site-packages/umap/umap_.py:1503: UserWarning: using precomputed metric; transform will be unavailable for new data and inverse_transform will be unavailable for all data
  "using precomputed metric; transform will be unavailable for new data and inverse_transform "


Computing decision regions ...
Time to calculate visualization for 50 samples: 10.10 sec


![random_forest](https://user-images.githubusercontent.com/30961397/78502477-a6ab5200-7761-11ea-8be3-e0b4c8e6a966.png)

In [21]:
deepview.close()

## Demo with DecisionTree

In [29]:
# --- Deep View Parameters ----
batch_size = 256
max_samples = 500
# the data can also be represented as a vector
sample_shape = (64,)
n = 10
lam = 0.65
resolution = 100
cmap = 'gist_ncar'

# the digit dataset is used, so classes are [0..9]
classes = np.arange(10)

In [30]:
pred_wrapper = DeepView.create_simple_wrapper(kn_neighbors.predict_proba)

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    sample_shape, n, lam, resolution, cmap)

# add data samples
n_samples = 200
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 77.18 sec


In [13]:
deepview.set_lambda(.4)
deepview.show()

Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...


In [9]:
deepview.close()

## Demo: KNN-Classifier

In [31]:
pred_wrapper = DeepView.create_simple_wrapper(kn_neighbors.predict_proba)

# create DeepView object
deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, 
                    sample_shape, n, lam, resolution, cmap)

# add data samples
n_samples = 200
sample_ids = np.random.choice(len(digits_X), n_samples)
X = np.array([ digits_X[i] for i in sample_ids ])
Y = np.array([ digits_y[i] for i in sample_ids ])

t0 = time.time()
deepview.add_samples(X, Y)
deepview.show()

print('Time to calculate visualization for %d samples: %.2f sec' % (n_samples, time.time() - t0))

Distance calculation 20.00 %
Distance calculation 40.00 %
Distance calculation 60.00 %
Distance calculation 80.00 %
Distance calculation 100.00 %
Embedding samples ...


/home/luca/Downloads/umap/umap/umap_.py:1428: UserWarning: Using precomputed metric; transform will be unavailable for new data
  warn("Using precomputed metric; transform will be unavailable for new data")


Computing decision regions ...
Time to calculate visualization for 200 samples: 79.15 sec


![knn](https://user-images.githubusercontent.com/30961397/78502740-dc046f80-7762-11ea-82cf-efc8251539db.png)


## Evaluating DeepView

In [12]:
from deepview.evaluate import evaluate

print('Evaluation of DeepView: %s\n' % deepview.title)
evaluate(deepview, X, Y)

Evaluation of DeepView: ResNet-20 - CIFAR10

orig labs, knn err: eucl / fish 0.29 / 0.46
orig labs, knn err in proj space: eucl / fish 0.8 / 0.29
classif labs, knn err: eucl / fish 0.26 / 0.5
classif labs, knn acc in proj space: eucl / fish 17.0 / 74.0


In [ ]:
deepview.close()